In [1]:
import numpy as np
import scipy.stats as sci
from collections import Counter

# a、g为均匀分布

In [2]:
D = lambda x: abs(x.real) + abs(x.imag)  # 距离
R = lambda x, X: sum(X == x)  # 到达x的次数

In [3]:
def SIS(t=30, n=100):
    '''sequencial importance sampling'''
    rejuvenation = 0
    X = np.array([[0] * n])  # 每一列是一条抽样，每一行是一次行走
    W = np.array([[1] * n])  # W的初始值为1
    U = np.array([[1] * n])  # U的初始值用不到，所以可以任取
    for i in range(1, t + 1):
        # 指数等可能取0123，分别是1,1j,-1,-1j，代表右上左下
        x_i = X[i - 1] + 1j**np.random.randint(0, 4, size=n)
        f_i = np.exp(-D(x_i) - R(x_i, X) / 2)
        u_i = f_i * 4
        w_i = W[i - 1] * u_i
        ESS = n / (1 + sci.variation(w_i)**2)  # coefficient of variation
        if ESS < n / 2 and i<t:
            # 多项分布重抽样
            multinomial = sci.multinomial.rvs(n, p=w_i/w_i.sum())
            l_tmp = []
            for j in range(n):
                l_tmp += [x_i[j]]*multinomial[j]
            x_rejuvenation = np.array(l_tmp)
            # 重置权重
            w_i = np.array([1] * n)
            rejuvenation += 1
        # 把新时刻的数据添加到纪录中
        X = np.append(X, np.reshape(x_i, (1, n)), axis=0)
        W = np.append(W, np.reshape(w_i, (1, n)), axis=0)
        U = np.append(U, np.reshape(u_i, (1, n)), axis=0)
    print(rejuvenation)
    return X, W[-1]

In [4]:
n = 10000
sample = SIS(30, n)
D_n = D(sample[0][-1])
W_n = sample[1]
M_n = np.array([max(Counter(list(sample[0][:, i])).values()) for i in range(n)])

26


In [5]:
ED = np.average(D_n, weights=W_n)
EM = np.average(M_n, weights=W_n)
print(f'E(D) = {ED}\nE(M) = {EM}')

E(D) = 1.7709844341503083
E(M) = 4.509474131820929


In [6]:
std_D = np.sqrt(np.average((D_n-ED)**2,weights=W_n))
std_M = np.sqrt(np.average((M_n-EM)**2,weights=W_n))
print(f'std(D) = {std_D}\nstd(M) = {std_M}')

std(D) = 1.4169736316173491
std(M) = 1.252075195318183


# c、SAW

这里直接暴力模拟，序贯抽样则使用R语言完成

In [7]:
def SAW(t=30):
    X = [0]
    for i in range(1,t+1):
        # 所有可能的move
        possiable = [1,-1,1j,-1j]
        for move in possiable[:]:
            if (X[i-1]+move) in X:# 避开走过的格点
                possiable.remove(move)
        c = len(possiable) # 可以走的格点
        if c == 0:
            return 'trapped'
        else:
            x_i = X[i-1] + possiable[np.random.randint(0,c)]
            X.append(x_i)
    return np.reshape(X,(1,t+1))

In [8]:
n = 20000
trapped = 0
l = np.zeros(shape=(1,31)) # 0-30共31个位置
for j in range(n):
    X = SAW()
    if isinstance(X,str):
        trapped+=1
    else:
        l = np.append(l,X,axis=0)
l = l[1:]
print(trapped/n)

0.20545


In [9]:
# D的均值与标准差
np.mean(D(l[:,-1])),np.std(D(l[:,-1]))

(11.085897677930904, 4.664178631880323)